# Signal files
First let's look at the available signal files

You can to this by running the command in the terminal:

`eosls /store/user/cmsdas/2024/short_exercises/tagging/`

You will find the following signal files:
- BulkGravToZZ_tree.root
- BulkGravTohh_tree.root
- BulkGravTohh_tree_noc.root
- ZprimeToTT_tree.root
- ZprimeToWW_tree.root

And QCD bacqground: 
- qcd-pythia_tree.root
- qcd-pythia_tree_SLIM.root
- qcd-pythia_tree_noc.root

# Let's start the exercise!
Open the file with `uproot` and convert the tree to a `pandas` dataframe.

We will start using the `BulkGravTohh` signal file but you can repeat the exercise with a different signal later.

In [ ]:
import uproot

filename_hh = 'root://cmseos.fnal.gov//store/user/cmsdas/2024/short_exercises/tagging/BulkGravTohh_tree_noc.root'
filename_qcd = 'root://cmseos.fnal.gov//store/user/cmsdas/2024/short_exercises/tagging/qcd-pythia_tree_SLIM.root'

uptree_hh = uproot.open(filename_hh)["Friends"]
uptree_qcd = uproot.open(filename_qcd)["Friends"]

branches = ['ak8_nn_HbbvsQCD',            # tagger DeepAK8
            'ak8_decorr_nn_ZHbbvsQCD',    # tagger DeepAK8-MD
            'ak8_doubleb',                # tagger
            'ak8_tau21',                  # tagger
            'ak8_ecfN2',                  # tagger
            'ak8_bestH',                  # tagger
            'ak8_sdmass',                 # tagger
            'ak8_pt',
            'ak8_eta',
            'gen_pt',
            'gen_eta']


print("get hh")
df_hh = uptree_hh.arrays(branches+['rewgt'], library="pd")
# print(df_hh)

print("get QCD")
df_qcd = uptree_qcd.arrays(branches+['genweight'], library="pd")
# print(df_qcd)

# Apply basic selections

In [ ]:
import pandas
import numpy as np

# mask higgs outside of fiducial cuts
mask_hh = (df_hh['gen_pt'] > 1000) & (df_hh['gen_pt'] < 1500) & (np.abs(df_hh['gen_eta']) < 2.4) & (df_hh['ak8_sdmass'] > 40) & (df_hh['ak8_sdmass'] < 200)
df_hh = df_hh[mask_hh]
df_hh['sample_weight'] = df_hh['rewgt']
df_hh['label'] = np.ones((len(df_hh)),dtype=int)

# mask qcd outside of fiducial cuts
mask_qcd = (df_qcd['gen_pt'] > 1000) & (df_qcd['gen_pt'] < 1500) & (np.abs(df_qcd['gen_eta']) < 2.4) & (df_qcd['ak8_sdmass'] > 40) & (df_qcd['ak8_sdmass'] < 200)
df_qcd = df_qcd[mask_qcd]
df_qcd['sample_weight'] = df_qcd['genweight']
df_qcd['label'] = np.zeros((len(df_qcd)),dtype=int)

# Let's plot the taggers for QCD and signal

In [ ]:
import matplotlib.pyplot as plt

plt.hist(df_hh['ak8_nn_HbbvsQCD'], bins=50, alpha=0.7, label='hh', histtype='step')
plt.hist(df_qcd['ak8_nn_HbbvsQCD'], bins=50, alpha=0.7, label='qcd', histtype='step')
plt.xlabel('ak8_nn_HbbvsQCD')
plt.yscale('log')
plt.legend()
plt.show()


# Concatenate signal and background
This is needed in the following

In [ ]:
df = pandas.concat([df_hh,df_qcd])

df

# Tagger performance

Plot the ROC and calculate the area under it for one of the tagging variables. We'll plot the true-positive rate (TPR) vs. the false-positive rate (FPR).

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
%matplotlib inline
fpr = {}
tpr = {}
threshold = {}
plt.figure()

tagger = 'ak8_sdmass'
fpr[tagger], tpr[tagger], threshold[tagger] = roc_curve(df['label'], df[tagger], sample_weight=df['sample_weight'])
myauc = auc(fpr[tagger], tpr[tagger]) #,reorder=True)
plt.plot(tpr[tagger],fpr[tagger],label='%s tagger, AUC = %.3f'%(tagger, myauc))

plt.legend(loc='best')
plt.semilogy()
plt.xlim([0, 1])
plt.ylim([1e-4, 1])
plt.xlabel("TPR");
plt.ylabel("FPR");

# Train your own tagger

In [ ]:
import keras
from keras.models import Model
from keras.layers import Input, Activation, Dense
import numpy as np

inputs = Input(shape=(2,), name = 'input')  
x = Dense(5, name = 'dense_1', activation = 'relu')(inputs)
outputs = Dense(1, name = 'output', activation='sigmoid')(x)

# create the model
model = Model(inputs=inputs, outputs=outputs)
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# print the model summary
model.summary()


### IGNORE THE RED WARNINGS!!!!

In [ ]:
x = np.vstack([df['ak8_sdmass'], df['ak8_tau21']]).T
print(x.shape)
y = df['label']
print(y.shape)
model.fit(x,y, epochs=1)
df['ak8_sdmass_tau21'] = model.predict(x)

In [ ]:
# now plot the ROC curve and calculate the AUC for this new ak8_sdmass_tau21 tagger

# Exercises

- Exercise 1: Plot all of the taggers on the same ROC curve
- Exercise 2 [done above, feel free to improve/play around]: Train your own custom "traditional" tagger based on mSD and tau21
- Exercise 3: Add your tagger to the ROC curve
- Exercise 4: Repeat with a different signal sample (W, Z, or top)
- Exercise 5 [advanced]: Understand the impact of different sets of inputs in the jet classification problem
- Exercise 6 [extra]: Look at the 'ak8_sdmass' distribution of signal and background after applying different cuts on the other taggers. Thoughts, ideas, concerns ?